<a href="https://colab.research.google.com/github/kodakallavenkataajaycharit25cs-hub/Ajay.K-repository/blob/master/Customer_Churn_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel('Telco_customer_churn.xlsx')

# Fix Total Charges
df['Total Charges'] = df['Total Charges'].astype(str).str.strip()
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')
df = df.dropna(subset=['Total Charges'])

# Binary 0/1
binary_cols = ['Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Paperless Billing']
for col in binary_cols:
    df[col] = df[col].astype(str).str.strip().map({'Yes': 1, 'No': 0}).astype('int64')

service_cols = ['Multiple Lines', 'Online Security', 'Online Backup',
                'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies']
for col in service_cols:
    df[col] = (df[col].astype(str).str.strip()
                          .replace({'No internet service': 'No', 'No phone service': 'No'})
                          .map({'Yes': 1, 'No': 0}).astype('int64'))

# One-hot encode
cat_cols = ['Gender', 'Internet Service', 'Contract', 'Payment Method']
for c in cat_cols:
    df[c] = df[c].astype(str).str.strip()

df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True, dtype='int64')

# Drop ID/geo/leaky
drop_cols = ['CustomerID','Count','Country','State','City','Zip Code','Lat Long',
             'Latitude','Longitude','Churn Label','Churn Reason','Churn Score','CLTV']
df_encoded = df_encoded.drop(columns=[c for c in drop_cols if c in df_encoded.columns])

print(df_encoded.shape)
print(df_encoded.dtypes.value_counts())
print('Nulls:', df_encoded.isnull().sum().sum())


(7032, 24)
int64      22
float64     2
Name: count, dtype: int64
Nulls: 0


In [60]:

X = df_encoded.drop(columns=['Churn Value'])
y = df_encoded['Churn Value']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

scalar = StandardScaler()
X_train_scaled = scalar.fit_transform(X_train)

X_test_scaled = scalar.transform(X_test)
print("Mean: ", X_train_scaled.mean())
print("Std: ", X_train_scaled.std())

model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

Mean:  -1.1615829071556421e-17
Std:  0.9999999999999999


In [61]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, classification_report,
    confusion_matrix, roc_curve
)


print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1-score :", f1_score(y_test, y_pred))
print("ROC AUC  :", roc_auc_score(y_test, y_proba))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred))


Accuracy : 0.8052594171997157
Precision: 0.6428571428571429
Recall   : 0.6016042780748663
F1-score : 0.6215469613259669
ROC AUC  : 0.8429332042594385

Confusion matrix:
 [[908 125]
 [149 225]]

Classification report:
               precision    recall  f1-score   support

           0       0.86      0.88      0.87      1033
           1       0.64      0.60      0.62       374

    accuracy                           0.81      1407
   macro avg       0.75      0.74      0.75      1407
weighted avg       0.80      0.81      0.80      1407

